In [41]:
from functools import partial, partialmethod, wraps
import json
from pathlib import Path
class Meta(type):
    def __new__(cls, clsname, bases, dct):
        print(f'Meta.__new__({cls}, {clsname}, {bases}, {dct})')
        methods = dct.copy()
        fields = []
        dir = Path.cwd() / '.objs'
        dir.mkdir(parents=True, exist_ok=True)
        for name, fn in dct.items():
            print(f'Checking: {name}: {fn}')
            if isinstance(fn, property):
                print(f'property: {name} -> {fn}')
                fields.append(name)
                field_name = f'_{name}'

                @wraps(fn)
                def wrapped_fn(self, name, field_name, fn, *args, **kwargs):
                    print(f'wrapped_fn: {name} ({field_name}): {fn}')
                    if not hasattr(self, field_name):
                        print(f'Computing: {name}')
                        assert isinstance(fn, property), fn
                        val = fn.fget(self, *args, **kwargs)
                        print(f'Computed: {name}={val}')
                        path = dir / name
                        print(f'Saving {name} to {path}')
                        with path.open('w') as f:
                            json.dump(val, f)
                        setattr(self, field_name, val)
                    else:
                        print(f'Lookup: {name}={getattr(self, field_name)}')
                    return getattr(self, field_name)

                applied = wraps(fn)(partialmethod(wrapped_fn, name=name, field_name=field_name, fn=fn))
                print(f'Setting method: {name}: {applied}')
                methods[name] = applied
            else:
                print(f'Skipping: {name}')
                    
        print(f'Fields: {fields}')
        orig_init = None
        if '__init__' in methods:
            orig_init = methods['__init__']

        def __init__(self, *args, **kwargs):
            print(f'__init__(self, {args}, {kwargs})')
            _kwargs = kwargs.copy()
            for field in fields:
                field_name = f'_{field}'
                if field in _kwargs:
                    val = _kwargs.pop(field)
                    print(f'Initializing {field_name}={val}')
                    setattr(self, field_name, val)
                else:
                    print(f'Skipping {field_name}')

            if orig_init:
                orig_init(self, *args, **_kwargs)
            
            print(f'Done with injected __init__: n={self.n}')
        
        methods['__init__'] = __init__

        def load(clz, dir, *args, **_kwargs):
            print(f'Calling: {clz}.load({dir}, {args}, {_kwargs})')
            kwargs = _kwargs.copy()
            dir = Path(dir)
            if not dir.exists():
                raise Exception(f"Can't load from non-existent directory {dir}")

            kwargs = {}    
            for field in fields:
                path = dir / field
                if path.exists():
                    with path.open('r') as f:
                        val = json.load(f)
                    if field in kwargs:
                        print(f'Loaded field {field}={val} being overridden by passed value {kwargs[field]}')
                    else:
                        kwargs[field] = val
            
            print(f'Loading with fields: {kwargs}; super {super(Meta, cls)}')
            self = clz.__new__(clz, clsname, bases, dct)
            
            __init__(self, *args, **kwargs)
            return self
        
        methods['load'] = classmethod(load)
        new = super(Meta, cls).__new__(cls, clsname, bases, methods)
        print(f'returning {new}: {methods}')

        return new 

In [42]:
class Foo(metaclass=Meta):
    def __init__(self, n):
        self.n = n
        print(f'Foo.__init__(self, n={n})')
        
    @property
    def a(self):
        return f'{self.n}-a'

    @property
    def b(self):
        return self.n * 2
    
    def __str__(f):
        print(f'Foo.__str__(n={f.n})')
        _a = getattr(f, '_a') if hasattr(f, '_a') else None
        _b = getattr(f, '_b') if hasattr(f, '_b') else None
        return f'Foo({f.a()} ({_a}), {f.b()} ({_b}))'

Meta.__new__(<class '__main__.Meta'>, Foo, (), {'__module__': '__main__', '__qualname__': 'Foo', '__init__': <function Foo.__init__ at 0x10c64f290>, 'a': <property object at 0x10c657bf0>, 'b': <property object at 0x10c657ad0>, '__str__': <function Foo.__str__ at 0x10c64fe60>})
Checking: __module__: __main__
Skipping: __module__
Checking: __qualname__: Foo
Skipping: __qualname__
Checking: __init__: <function Foo.__init__ at 0x10c64f290>
Skipping: __init__
Checking: a: <property object at 0x10c657bf0>
property: a -> <property object at 0x10c657bf0>
Setting method: a: functools.partialmethod(<function Meta.__new__.<locals>.wrapped_fn at 0x10c649b00>, , name='a', field_name='_a', fn=<property object at 0x10c657bf0>)
Checking: b: <property object at 0x10c657ad0>
property: b -> <property object at 0x10c657ad0>
Setting method: b: functools.partialmethod(<function Meta.__new__.<locals>.wrapped_fn at 0x10c6493b0>, , name='b', field_name='_b', fn=<property object at 0x10c657ad0>)
Checking: __str

In [44]:
f = Foo(111)

__init__(self, (111,), {})
Skipping _a
Skipping _b
Foo.__init__(self, n=111)
Done with injected __init__: n=111


In [48]:
hasattr(f, '_a'), hasattr(f, '_b'), f.a, f.b

(False,
 False,
 functools.partial(<bound method Meta.__new__.<locals>.wrapped_fn of <__main__.Foo object at 0x10c66bc50>>, name='a', field_name='_a', fn=<property object at 0x10c657bf0>),
 functools.partial(<bound method Meta.__new__.<locals>.wrapped_fn of <__main__.Foo object at 0x10c66bc50>>, name='b', field_name='_b', fn=<property object at 0x10c657ad0>))

In [49]:
f.a()

wrapped_fn: a (_a): <property object at 0x10c657bf0>
Computing: a
Computed: a=111-a
Saving a to /Users/ryan/c/ur/.objs/a


'111-a'

In [50]:
foo = Foo.load('.objs', 222); foo

Calling: <class '__main__.Foo'>.load(.objs, (222,), {})
Loading with fields: {'a': '111-a'}; super <super: <class 'Meta'>, <Meta object>>
__init__(self, (222,), {'a': '111-a'})
Initializing _a=111-a
Skipping _b
Foo.__init__(self, n=222)
Done with injected __init__: n=222


In [51]:
foo.a()

wrapped_fn: a (_a): <property object at 0x10c657bf0>
Lookup: a=111-a


'111-a'

In [52]:
foo.n

222

In [53]:
foo._b

AttributeError: 'Foo' object has no attribute '_b'

In [7]:
foo.a(), foo._a

wrapped_fn: a (_b): <property object at 0x10b5815f0>
Lookup: a=None


(None, None)

In [ ]:
str(foo)

In [24]:
hasattr(f, '_a')

False

In [20]:
f.a()

wrapped_fn: a (_b): <property object at 0x10b5db1d0>
Lookup: a=111-a


'111-a'

In [ ]:
f.id, f._a, f._b

In [ ]:
f.a, f.b

In [ ]:
class Bar:
    def __init__(self, n):
        self.n = n

In [ ]:
bar = Bar.__new__(Bar, 'Bar', ); bar

In [ ]:
Bar.__init__(bar, 12)

In [ ]:
bar.n